# ECG arrhythmia classification using CNN

In [20]:
pwd

'/home/wsuser/work'

In [22]:
!pip install keras==2.2.4
!pip install tensorflow

### Importing Neccessary Libraries

In [23]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


### Image Data Agumentation

In [24]:
#setting parameter for Image Data agumentation to the traing data
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [25]:
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [26]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_112eebfd30a442d9bdc6ed17f9e9f866 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='cYktFW4G0vDfgElW48YxgJQbVbOZoBC-nx2SD7VXh2Vs',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_112eebfd30a442d9bdc6ed17f9e9f866.get_object(Bucket='ecg-donotdelete-pr-l134tswveqaqri', Key='Dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [27]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [28]:
pwd

'/home/wsuser/work'

In [29]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset/train')

In [30]:
#performing data agumentation to train data
x_train=train_datagen.flow_from_directory('/home/wsuser/work/Dataset/train'
                                          ,target_size=(64,64),batch_size=32,class_mode='categorical')
#performing data agumentation to test data
x_test=test_datagen.flow_from_directory('/home/wsuser/work/Dataset/test'
                                        ,target_size=(64,64),batch_size=32,class_mode='categorical')

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [31]:
print(x_train.class_indices)#checking the number of classes

{'Left Bundle Branch Block': 0, 'Normal': 1, 'Premature Atrial Contraction': 2, 'Premature Ventricular Contractions': 3, 'Right Bundle Branch Block': 4, 'Ventricular Fibrillation': 5}


In [32]:
from collections import Counter as c
c(x_train.labels)

Counter({0: 504, 1: 7346, 2: 2054, 3: 2759, 4: 2239, 5: 439})

### Creating the model

In [33]:
# create model
model=Sequential()
# adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))#convolutional layer
model.add(MaxPooling2D(pool_size=(2,2))) #MaxPooling2D-for downsampling the input

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(32))#deeply connected neural network layers.
model.add(Dense(6,activation='softmax'))#output layer with 6 neurons


In [34]:
model.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                200736    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

### Compiling the model

In [35]:
# Compile model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

### Fitting the model

In [36]:
# Fit the model 
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train),
                    epochs=10, validation_data=x_test,validation_steps = len(x_test))

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
480/480 [==============================] - 69s 142ms/step - loss: 0.9874 - accuracy: 0.6579 - val_loss: 0.3795 - val_accuracy: 0.8819
Epoch 2/10
480/480 [==============================] - 68s 142ms/step - loss: 0.2543 - accuracy: 0.9230 - val_loss: 0.5231 - val_accuracy: 0.8595
Epoch 3/10
480/480 [==============================] - 70s 145ms/step - loss: 0.2087 - accuracy: 0.9397 - val_loss: 0.4839 - val_accuracy: 0.8708
Epoch 4/10
480/480 [==============================] - 71s 149ms/step - loss: 0.1763 - accuracy: 0.9459 - val_loss: 0.3286 - val_accuracy: 0.8944
Epoch 5/10
480/480 [==============================] - 70s 145ms/step - loss: 0.1488 - accuracy: 0.9564 - val_loss: 0.2883 - val_accuracy: 0.9172
Epoch 6/10
480/480 [==============================] - 68s 143ms/step - loss: 0.1226 - accuracy: 0.9617 - val_loss: 0.2982 - val_accuracy: 0.9272
Epoch 7/10
480/480 [==============================] - 70s 146ms/step - loss: 0.1262 - accuracy: 0.9606 - val_loss: 0.4205 - val_ac

In [37]:
#model.fit_gene rator(x_train,epochs=10,validation_data=x_test)

### Saving our model

In [38]:
# Save the model
from tensorflow.keras.models import load_model
model.save('ibm_ECG.h5')

In [39]:
!tar -zcvf ECG-Image-based-heartbeat-classification-model_new.tgz ibm_ECG.h5

ibm_ECG.h5


In [40]:
ls -1

Dataset/
ECG-Image-based-heartbeat-classification-model_new.tgz
ibm_ECG.h5


In [41]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 28.2 MB/s eta 0:00:01


In [42]:
# Replace the credentials that you got from watson machine learning service
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"cYktFW4G0vDfgElW48YxgJQbVbOZoBC-nx2SD7VXh2Vs"
                  }
client = APIClient(wml_credentials)


In [43]:
client = APIClient(wml_credentials)

In [44]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [45]:
space_uid = guid_from_space_name(client, 'ECG_deploy')
print("Space UID = "+ space_uid)

Space UID = 09c29b80-1ab3-420c-9e73-037ee0d90470


In [46]:
client.set.default_space(space_uid)

'SUCCESS'

In [47]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [48]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.8-horovod")
software_spec_uid

'217c16f6-178f-56bf-824a-b19f20564c49'

In [49]:
model_details = client.repository.store_model(model='ECG-Image-based-heartbeat-classification-model_new.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                              )
model_id = client.repository.get_model_uid(model_details)
    

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html
This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/ibm_watson_machine_learning/repository.py:1458: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [50]:
model_id

'f937ba8c-08e4-4edc-96af-c6343219f68b'

In [51]:
client.repository.download(model_id, 'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [52]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [53]:
model = load_model("ibm_ECG.h5")